# nflfastR quick analysis (EPA & CPOE per passer)

This notebook downloads the nflfastR play-by-play CSV, computes average EPA and CPOE by passer, prints the top quarterbacks, and saves the summary to CSV.

Notes:
- The file can be large. Use the Parameters cell to enable chunked aggregation if you have limited memory.
- Edit the `year` variable in the Parameters cell and re-run.

In [ ]:
# Parameters — edit these and re-run the notebook
# year: season to download/analyze
year = 2025
# If True, always re-download the remote CSV even when a cached copy exists
force_redownload = False
# How old (in hours) a cached file can be before we re-download (used by the main cell)
cache_hours = 24

# Chunking and aggregation knobs
# When True the notebook will stream the CSV in chunks and aggregate incrementally
use_chunked_aggregation = False
# Number of rows per chunk when chunking is enabled
chunksize = 200_000
# Minimum number of plays a passer must have to appear in the final summary
min_plays = 50


In [ ]:
# OPTIONAL: install pandas in this environment if needed (uncomment to run)
# !pip install pandas

In [ ]:
import pandas as pd
import os
import time
from pathlib import Path
# Optional: use tqdm for nicer progress bars if available, but import dynamically to avoid static analyzer warnings
try:
    import importlib, importlib.util
    if importlib.util.find_spec('tqdm.auto') is not None:
        _tqdm = importlib.import_module('tqdm.auto').tqdm
    else:
        _tqdm = None
except Exception:
    _tqdm = None

# Build remote URL and local paths using the Parameters cell
url = f"https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.csv.gz"
out_dir = Path('data')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / f'play_by_play_{year}.csv.gz'

def download_with_requests(u, dest, attempts=3, backoff=3):
    try:
        import requests
    except Exception:
        return False, 'missing requests library'
    last = None
    for i in range(1, attempts+1):
        try:
            with requests.get(u, stream=True, timeout=30) as r:
                r.raise_for_status()
                with open(dest, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=1024*1024):
                        if chunk:
                            f.write(chunk)
            return True, None
        except Exception as e:
            last = str(e)
            print(f'Download attempt {i} failed: {e}')
            time.sleep(backoff * i)
    return False, last

def download_with_urllib(u, dest):
    try:
        import urllib.request
        urllib.request.urlretrieve(u, dest)
        return True, None
    except Exception as e:
        return False, str(e)

# Determine whether to download
need_download = force_redownload or (not out_path.exists())
if out_path.exists() and not force_redownload:
    age = time.time() - out_path.stat().st_mtime
    if age < cache_hours * 3600:
        need_download = False

if need_download:
    print('Downloading play-by-play (may take a few minutes)...')
    ok, err = download_with_requests(url, out_path)
    if not ok:
        print('requests download failed, falling back to urllib:', err)
        ok2, err2 = download_with_urllib(url, out_path)
        if not ok2:
            raise RuntimeError(f'All download attempts failed. requests error={err}; urllib error={err2}')
    print('Download complete:', out_path)
else:
    print('Using cached file:', out_path)

print('Reading CSV with pandas (memory-friendly)')
needed_cols = ['play_id', 'passer_player_name', 'epa', 'cpoe']
dtypes = {'play_id': 'Int64', 'passer_player_name': 'string', 'epa': 'float32', 'cpoe': 'float32'}
# Quick header check
sample_hdr = pd.read_csv(out_path, compression='gzip', nrows=0)
missing = set(needed_cols) - set(sample_hdr.columns)
if missing:
    raise RuntimeError(f'Missing required columns in CSV: {missing}')

# Decide on chunking
do_chunked = bool(use_chunked_aggregation) or (chunksize is not None)
if chunksize in (0, None):
    do_chunked = bool(use_chunked_aggregation)

if do_chunked:
    print(f'Using chunked aggregation with chunksize={chunksize}')
    acc = {}  # passer -> [sum_epa, sum_cpoe, plays]
    # Iterate chunks with a progress indicator (tqdm if available)
    reader = pd.read_csv(out_path, compression='gzip', usecols=needed_cols, dtype=dtypes, chunksize=chunksize)
    if _tqdm is not None:
        # Try to estimate total number of chunks from file size (best-effort)
        try:
            total_bytes = out_path.stat().st_size
            avg_row_bytes = 100  # rough estimate
            est_rows = max(1, int(total_bytes / max(1, avg_row_bytes)))
            est_chunks = max(1, int(est_rows / max(1, chunksize)))
        except Exception:
            est_chunks = None
        iterator = _tqdm(reader, total=est_chunks, desc='chunks')
    else:
        iterator = reader
    chunk_i = 0
    for chunk in iterator:
        chunk_i += 1
        if _tqdm is None and chunk_i % 10 == 0:
            print(f'Processed {chunk_i} chunks...')
        chunk = chunk.dropna(subset=['passer_player_name'])
        grp = chunk.groupby('passer_player_name', dropna=True).agg(sum_epa=('epa', 'sum'), sum_cpoe=('cpoe', 'sum'), plays=('play_id', 'count'))
        for name, row in grp.iterrows():
            if name in acc:
                acc[name][0] += float(row['sum_epa'])
                acc[name][1] += float(row['sum_cpoe'])
                acc[name][2] += int(row['plays'])
            else:
                acc[name] = [float(row['sum_epa']), float(row['sum_cpoe']), int(row['plays'])]
    agg_df = pd.DataFrame.from_dict(acc, orient='index', columns=['sum_epa', 'sum_cpoe', 'plays'])
    agg_df.index.name = 'passer_player_name'
    agg_df = agg_df.reset_index()
    agg_df['avg_epa'] = agg_df['sum_epa'] / agg_df['plays']
    agg_df['avg_cpoe'] = agg_df['sum_cpoe'] / agg_df['plays']
    summary = (
        agg_df.loc[lambda d: d['plays'] >= min_plays]
              .sort_values('avg_epa', ascending=False)
              .head(10)
              .loc[:, ['avg_epa', 'avg_cpoe', 'plays']]
    )
else:
    try:
        df = pd.read_csv(out_path, compression='gzip', usecols=needed_cols, dtype=dtypes)
    except Exception as e:
        print('Optimized read failed, falling back to full read:', e)
        df = pd.read_csv(out_path, compression='gzip', low_memory=False)
    print(f'Loaded {len(df):,} plays (rows)')
    summary = (
        df.groupby('passer_player_name', dropna=True)
          .agg(avg_epa=('epa', 'mean'),
               avg_cpoe=('cpoe', 'mean'),
               plays=('play_id', 'count'))
          .loc[lambda d: d['plays'] >= min_plays]
          .sort_values('avg_epa', ascending=False)
          .head(10)
    )

print(summary)
out_summary = out_dir / f'epa_cpoe_summary_{year}.csv'
summary.to_csv(out_summary, index=True)
print(f'Wrote {out_summary}')
